In [1]:
from datasets import load_dataset


dataset = load_dataset("liar")   

'''
User: Mels Habold
Data downloaded from https://huggingface.co/datasets/liar
Based on research from https://arxiv.org/abs/1705.00648
And an article posted in https://www.analyticssteps.com/blogs/detection-fake-and-false-news-text-analysis-approaches-and-cnn-deep-learning-model 
''' 

Found cached dataset liar (C:/Users/Mels/.cache/huggingface/datasets/liar/default/1.0.0/479463e757b7991eed50ffa7504d7788d6218631a484442e2098dabbf3b44514)


  0%|          | 0/3 [00:00<?, ?it/s]

'\nUser: Mels Habold\nData downloaded from https://huggingface.co/datasets/liar\nBased on research from https://arxiv.org/abs/1705.00648\nAnd an article posted in https://www.analyticssteps.com/blogs/detection-fake-and-false-news-text-analysis-approaches-and-cnn-deep-learning-model \n'

## The LIAR Dataset

We consider six fine-grained labels for the truthfulness ratings: 
pants-fire, false, barelytrue, half-true, mostly-true, and true.

It consists of three sets: The training set is used to fit the model's parameters, while the validation set is used to tune the model's hyperparameters (e.g., learning rate, number of layers, etc.) and to monitor the model's performance during training. The test set is used to evaluate the final performance of the model after it has been trained and fine-tuned using the training and validation sets.

The next code allows us to split the data into 3 types:
False, in between, True

In [2]:
import numpy as np

if True:
    train_label = np.array(dataset["train"]['label'])
    test_label = np.array(dataset["test"]['label'])
    validation_label = np.array(dataset["validation"]['label'])
    
    train_label = np.where(train_label==1, 0, train_label)
    train_label = np.where(train_label==2, 1, train_label)
    train_label = np.where(train_label==3, 1, train_label)
    train_label = np.where(train_label==4, 2, train_label)
    train_label = np.where(train_label==5, 2, train_label)
    
    test_label = np.where(test_label==1, 0, test_label)
    test_label = np.where(test_label==2, 1, test_label)
    test_label = np.where(test_label==3, 1, test_label)
    test_label = np.where(test_label==4, 2, test_label)
    test_label = np.where(test_label==5, 2, test_label)
    
    validation_label = np.where(validation_label==1, 0, validation_label)
    validation_label = np.where(validation_label==2, 1, validation_label)
    validation_label = np.where(validation_label==3, 1, validation_label)
    validation_label = np.where(validation_label==4, 2, validation_label)
    validation_label = np.where(validation_label==5, 2, validation_label)

else:
    train_label = dataset["train"]['label']
    test_label = dataset["test"]['label']
    validation_label = dataset["validation"]['label']
    
print(np.unique(train_label))

[0 1 2]


## Model building

The first part of the code imports the necessary libraries and defines the hyperparameters. Then, it loads the training data from a dataset object and tokenizes the text using the Tokenizer class from Keras. The Tokenizer class is used to convert text to sequences of integers. The num_words parameter specifies the maximum number of words to keep, based on word frequency. The next step is to pad the sequences so that all input data have the same length using the pad_sequences function from Keras.

After the preprocessing steps, the code loads a pre-trained Word2Vec model using the Gensim library. Then, it creates an embedding matrix using the Word2Vec model to map each word in the input sequences to a corresponding vector in the embedding space. The embedding matrix is used to initialize the embedding layer in the CNN.

In [3]:
#import the necessary libraries
import numpy as np
import tensorflow as tf
import warnings
 
warnings.filterwarnings(action = 'ignore')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import *
from keras.models import Model
from keras.utils import to_categorical
import gensim
from gensim.models import Word2Vec

#define the hyperparameters
max_words = 100                 # max number of words in a statement
embedding_dim = 50              # dimension of the word vector
output_dim = tf.unique(train_label)[0].shape[0]  # number of output labels

#tokenize the statements
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dataset["train"]['statement'])
sequences_statement = tokenizer.texts_to_sequences(dataset["train"]['statement'])

#tokenize the statements
tokenizer.fit_on_texts(dataset["train"]['subject'])
sequences_subject = tokenizer.texts_to_sequences(dataset["train"]['subject'])

#pad the sequences
x_data_subject = pad_sequences(sequences_statement, maxlen=max_words)
x_data_statement = pad_sequences(sequences_subject, maxlen=max_words)
y_data = to_categorical(train_label, num_classes=output_dim)

x_data = [x_data_subject, x_data_statement]
sequences_combined = sequences_statement + sequences_subject

# Load the pre-trained Word2Vec model
#train Word2Vec model
sentences = dataset["train"]['statement'] + dataset["train"]['subject']
sentences = [sentence.split() for sentence in sentences]
w2v_model = Word2Vec(sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Create the embedding matrix
embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_words:
        try:
            embedding_vector = w2v_model.wv[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            pass

# Create the embedding layer
embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim,
                            weights=[embedding_matrix], input_length=max_words)

## CNN Architecture

1. Embedding layer: This layer maps the integer-encoded vocabulary to dense vectors of fixed size, in this case, an embedding vector of dimension embedding_dim for each word. The input_dim parameter specifies the size of the vocabulary, which is set to max_words in this code. The output_dim parameter specifies the size of the embedding vector.

2. Bidirectional layer that wraps around the LSTM layer. The Bidirectional layer takes the LSTM layer as input and creates two copies of it, one for processing the text forward and the other for processing it backward. The outputs from these two layers are then concatenated and passed to the next layer.

3. Convolutional layer: This layer applies num_filters filters of size kernel_size to the input sequence. The filters slide over the input sequence, producing a feature map of convolved features. The activation parameter is set to ReLU, which means that the output of the layer will be the rectified linear activation function of the convolved features.

4. Max pooling layer: This layer downsamples the convolved features by taking the maximum value of each non-overlapping pool_size-sized segment of the feature map.

5. Flatten layer: This layer flattens the output of the previous layer to a 1D array.

6. Dense layer: This layer is a fully connected layer with hidden_dim units. The activation function used is ReLU, which applies the rectified linear activation function to the output.

7. Dropout layers randomly drop out some of the neurons during training, which can help prevent overfitting. This can be added using the Dropout layer in Keras.

8. Output layer: This layer is a fully connected layer with output_dim units, which corresponds to the number of output classes. The activation function used is softmax, which produces a probability distribution over the classes.

In summary, the CNN takes as input a sequence of integers representing the words in a text, and passes it through an embedding layer to obtain dense vectors. Then, the convolutional layer applies a set of filters to the sequence, producing a feature map that is downsampled by the max pooling layer. The flatten layer converts the output of the max pooling layer into a 1D array, which is passed through a dense layer and finally an output layer to produce a probability distribution over the classes.

In [4]:
#define the hyperparameters
num_filters = 128            # number of convolutional filters
kernel_size = 3              # size of convolutional kernel
pool_size = 2                # size of pooling window
hidden_dim = 50              # dimension of fully connected layers

# Define the two inputs
input_subject = Input(shape=(max_words,), dtype='int32')
input_statement = Input(shape=(max_words,), dtype='int32')

# Define the embedding layer
#embedding_layer = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_words)

# Apply the embedding layer to both inputs
embedded_subject = embedding_layer(input_subject)
embedded_statement = embedding_layer(input_statement)

# Merge the two embedded inputs using the Concatenate layer
merged = Concatenate(axis=-1)([embedded_subject, embedded_statement])

# Define the model
conv_layer = Conv1D(filters=num_filters, kernel_size=kernel_size, activation='softmax')(merged)
pool_layer = MaxPooling1D(pool_size=pool_size)(conv_layer)
flatten_layer = Flatten()(pool_layer)
hidden_layer = Dense(hidden_dim, activation='relu')(flatten_layer)
output = Dense(output_dim, activation='softmax')(hidden_layer)

model = Model(inputs=[input_subject, input_statement], outputs=output)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 100, 50)      5000        ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 concatenate (Concatenate)      (None, 100, 100)     0           ['embedding[0][0]',          

## Training

The second part of the code trains the CNN using the fit function in Keras. The batch_size parameter specifies the number of samples processed in one batch, and the epochs parameter specifies the number of training epochs. The compile function is used to configure the learning process, including the optimizer, loss function, and evaluation metric. The adam optimizer is used, and the categorical_crossentropy loss function is used since this is a multi-class classification problem. The evaluation metric is accuracy.

After training, the model is evaluated using the evaluate function in Keras, and the test loss and accuracy are printed.

In [5]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3)

#compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#train the model
#model.fit(x_data, y_data, batch_size=32, epochs=10)
model.fit(x_data, y_data, validation_split=0.1, batch_size=32, epochs=10, callbacks=[early_stop])

Epoch 1/10
289/289 [==============================] - 7s 22ms/step - loss: 1.0799 - accuracy: 0.3966 - val_loss: 1.0773 - val_accuracy: 0.4119
Epoch 2/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0763 - accuracy: 0.3987 - val_loss: 1.0749 - val_accuracy: 0.4119
Epoch 3/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0733 - accuracy: 0.4039 - val_loss: 1.0697 - val_accuracy: 0.4148
Epoch 4/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0684 - accuracy: 0.4101 - val_loss: 1.0635 - val_accuracy: 0.4382
Epoch 5/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0596 - accuracy: 0.4266 - val_loss: 1.0650 - val_accuracy: 0.4070
Epoch 6/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0540 - accuracy: 0.4370 - val_loss: 1.0647 - val_accuracy: 0.4158
Epoch 7/10
289/289 [==============================] - 6s 22ms/step - loss: 1.0483 - accuracy: 0.4442 - val_loss: 1.0570 - val_accuracy: 0.4372

## Evaluating Precise

In [ ]:
# create the test data
tokenizer.fit_on_texts(dataset["test"]['statement'])
sequences_statement = tokenizer.texts_to_sequences(dataset["test"]['statement'])

tokenizer.fit_on_texts(dataset["test"]['subject'])
sequences_subject = tokenizer.texts_to_sequences(dataset["test"]['subject'])

#pad the sequences
x_test = [pad_sequences(sequences_statement, maxlen=max_words), pad_sequences(sequences_subject, maxlen=max_words)]
y_test = to_categorical(test_label, num_classes=output_dim)

#evaluate the model
loss, accuracy = model.evaluate(x_test, y_test, batch_size=32)

#print the results
print('Test loss:', loss)
print('Test accuracy:', accuracy)

 1/41 [..............................] - ETA: 1s - loss: 1.1487 - accuracy: 0.3438